In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.preprocessing import Imputer , Normalizer , scale, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.metrics import make_scorer, accuracy_score, classification_report, confusion_matrix, mean_squared_error
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas import get_dummies
import xgboost as xgb
import scipy
import math
import json
import sys
import csv
import os
import tqdm
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Dropout, GRU, Bidirectional, Flatten, Embedding
from keras.optimizers import SGD
from tqdm import tqdm_notebook
from nltk.corpus import stopwords
import string
from collections import Counter
from string import punctuation
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from gensim.models import Word2Vec
import re
from string import digits
import operator
import gc

Using TensorFlow backend.


In [2]:
def build_vocab(texts, vocab):
    #sentences = texts.apply(lambda x: x.split()).values
    for sentence in texts:
        for word in sentence.split(' '):
            if word == '' or word == ' ':
                continue
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def load_embed(file):
    def get_coefs(word, *arr):
        return word, np.asarray(arr, dtype='float32')
    if file == 'wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o) > 100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
    return embeddings_index

def load_doc(filename):
    lines = [line.rstrip('\n') for line in open(filename)]
    result = []
    for line in lines:
        result.append(line)
    return result

def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]
    return unknown_words

def add_lower(embedding, vocab):
    count = 0
    for word in vocab:
        if word in embedding and word.lower() not in embedding:  
            embedding[word.lower()] = embedding[word]
            count += 1
    print(f"Added {count} words to embedding")
    
def known_contractions(embed):
    known = []
    for contract in contraction_mapping:
        if contract in embed:
            known.append(contract)
    return known

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

def unknown_punct(embed, punct):
    unknown = ''
    for p in punct:
        if p not in embed:
            unknown += p
            unknown += ' '
    return unknown

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

In [3]:
questions = load_doc("questions.txt")
answers = load_doc("answers.txt")
list_of_tuples = list(zip(questions, answers))
dataset = pd.DataFrame(list_of_tuples, columns = ['Questions', 'Answers'])

In [4]:
dataset.head()

,Questions,Answers
0,What is ICTS learning analytics ?,"It is a modern, smart and focussed virtual env..."
1,Is this a degree program ?,"No, this is a short term training programme. I..."
2,Is the certification given by any industry or ...,"Yes, certification is given by ICTS Pvt. Ltd. ..."
3,Is this completely theoretical or some hands o...,Some hands on training will be available. Howe...
4,Is this an online programme or classroom conta...,It is an online program that mimics classroom ...


In [5]:
glove = 'glove.6B.300d.txt'
wiki_news = 'wiki-news-300d-1M.vec'

print("Extracting GloVe embedding")
embed_glove = load_embed(glove)
print("Extracting FastText embedding")
embed_fasttext = load_embed(wiki_news)

Extracting GloVe embedding
Extracting FastText embedding


In [6]:
vocab = {}
vocab = build_vocab(dataset['Questions'], vocab)
vocab = build_vocab(dataset['Answers'], vocab)

In [7]:
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Glove : 
Found embeddings for 74.19% of vocab
Found embeddings for  81.10% of all text
FastText : 
Found embeddings for 91.40% of vocab
Found embeddings for  95.05% of all text


In [8]:
dataset['lowered_question'] = dataset['Questions'].apply(lambda x: x.lower())
dataset['lowered_answer'] = dataset['Answers'].apply(lambda x: x.lower())

In [9]:
dataset.head()

,Questions,Answers,lowered_question,lowered_answer
0,What is ICTS learning analytics ?,"It is a modern, smart and focussed virtual env...",what is icts learning analytics ?,"it is a modern, smart and focussed virtual env..."
1,Is this a degree program ?,"No, this is a short term training programme. I...",is this a degree program ?,"no, this is a short term training programme. i..."
2,Is the certification given by any industry or ...,"Yes, certification is given by ICTS Pvt. Ltd. ...",is the certification given by any industry or ...,"yes, certification is given by icts pvt. ltd. ..."
3,Is this completely theoretical or some hands o...,Some hands on training will be available. Howe...,is this completely theoretical or some hands o...,some hands on training will be available. howe...
4,Is this an online programme or classroom conta...,It is an online program that mimics classroom ...,is this an online programme or classroom conta...,it is an online program that mimics classroom ...


In [10]:
vocab_low = {}
vocab_low = build_vocab(dataset['lowered_question'], vocab_low)
vocab_low = build_vocab(dataset['lowered_answer'], vocab_low)

In [11]:
print("Glove : ")
oov_glove = check_coverage(vocab_low, embed_glove)
print("FastText : ")
oov_fasttext = check_coverage(vocab_low, embed_fasttext)

Glove : 
Found embeddings for 85.02% of vocab
Found embeddings for  90.99% of all text
FastText : 
Found embeddings for 91.01% of vocab
Found embeddings for  94.88% of all text


In [12]:
print("Glove : ")
add_lower(embed_glove, vocab)
print("FastText : ")
add_lower(embed_fasttext, vocab)

Glove : 
Added 0 words to embedding
FastText : 
Added 1 words to embedding


In [13]:
print("Glove : ")
oov_glove = check_coverage(vocab_low, embed_glove)
print("FastText : ")
oov_fasttext = check_coverage(vocab_low, embed_fasttext)

Glove : 
Found embeddings for 85.02% of vocab
Found embeddings for  90.99% of all text
FastText : 
Found embeddings for 91.39% of vocab
Found embeddings for  95.23% of all text


In [14]:
oov_glove

[('support.', 3),
 ('programme.', 3),
 ('students.', 3),
 ('programme?', 3),
 ('candidate.', 2),
 ('yes,', 2),
 ('there?', 2),
 ('platform.', 1),
 ('laptop.', 1),
 ('journals.', 1),
 ('participants.', 1),
 ('limited,', 1),
 ('program.', 1),
 ('possible.', 1),
 ('experts.', 1),
 ('lectures.', 1),
 ('90%', 1),
 ('domains.', 1),
 ('participation.', 1),
 ('organisations.', 1),
 ('placements.', 1),
 ('therefore,', 1),
 ('disciplines.', 1),
 ('engineering,', 1),
 ('weekend.', 1),
 ('tutorials.', 1),
 ('driven.', 1),
 ('available.', 1),
 ('leads.', 1),
 ('analytics.', 1),
 ('hours/credits', 1),
 ('no,', 1),
 ('information.', 1),
 ('http://www.icts-learninganalytics.com', 1),
 ('modern,', 1),
 ('others?', 1),
 ('process?', 1),
 ('criteria?', 1),
 ('provided?', 1),
 ('available?', 1)]

In [15]:
contraction_mapping = {"ain't": "is not", "aren't": "are not", "http://www.icts-learninganalytics.com": "the official site", "can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [16]:
print("- Known Contractions -")
print("   Glove :")
print(known_contractions(embed_glove))
print("   FastText :")
print(known_contractions(embed_fasttext))

- Known Contractions -
   Glove :
["'cause", "ma'am", "o'clock"]
   FastText :
[]


In [17]:
dataset['treated_question'] = dataset['lowered_question'].apply(lambda x: clean_contractions(x, contraction_mapping))
dataset['treated_answer'] = dataset['lowered_answer'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [18]:
vocab = {}
vocab = build_vocab(dataset['treated_question'], vocab)
vocab = build_vocab(dataset['treated_answer'], vocab)
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Glove : 
Found embeddings for 85.45% of vocab
Found embeddings for  91.20% of all text
FastText : 
Found embeddings for 91.79% of vocab
Found embeddings for  95.42% of all text


In [19]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [20]:
print("Glove :")
print(unknown_punct(embed_glove, punct))
print("FastText :")
print(unknown_punct(embed_fasttext, punct))

Glove :
“ ” ’ ∞ θ ÷ α • à − β ∅ ³ π ‘ ₹ ´ ° £ € × ™ √ ² — – 
FastText :
_ ` 


In [21]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [22]:
dataset['treated_question'] = dataset['treated_question'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
dataset['treated_answer'] = dataset['treated_answer'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [23]:
vocab = {}
vocab = build_vocab(dataset['treated_question'], vocab)
vocab = build_vocab(dataset['treated_answer'], vocab)
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Glove : 
Found embeddings for 100.00% of vocab
Found embeddings for  100.00% of all text
FastText : 
Found embeddings for 100.00% of vocab
Found embeddings for  100.00% of all text


In [24]:
oov_fasttext

[]

In [25]:
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

In [26]:
dataset['treated_question'] = dataset['treated_question'].apply(lambda x: correct_spelling(x, mispell_dict))
dataset['treated_answer'] = dataset['treated_answer'].apply(lambda x: correct_spelling(x, mispell_dict))

In [27]:
vocab = {}
vocab = build_vocab(dataset['treated_question'], vocab)
vocab = build_vocab(dataset['treated_answer'], vocab)
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Glove : 
Found embeddings for 100.00% of vocab
Found embeddings for  100.00% of all text
FastText : 
Found embeddings for 100.00% of vocab
Found embeddings for  100.00% of all text


In [28]:
dataset.head()

,Questions,Answers,lowered_question,lowered_answer,treated_question,treated_answer
0,What is ICTS learning analytics ?,"It is a modern, smart and focussed virtual env...",what is icts learning analytics ?,"it is a modern, smart and focussed virtual env...",what is icts learning analytics ?,"it is a modern , smart and focussed virtual e..."
1,Is this a degree program ?,"No, this is a short term training programme. I...",is this a degree program ?,"no, this is a short term training programme. i...",is this a degree program ?,"no , this is a short term training programme ..."
2,Is the certification given by any industry or ...,"Yes, certification is given by ICTS Pvt. Ltd. ...",is the certification given by any industry or ...,"yes, certification is given by icts pvt. ltd. ...",is the certification given by any industry or ...,"yes , certification is given by icts pvt . l..."
3,Is this completely theoretical or some hands o...,Some hands on training will be available. Howe...,is this completely theoretical or some hands o...,some hands on training will be available. howe...,is this completely theoretical or some hands o...,some hands on training will be available . ho...
4,Is this an online programme or classroom conta...,It is an online program that mimics classroom ...,is this an online programme or classroom conta...,it is an online program that mimics classroom ...,is this an online programme or classroom conta...,it is an online program that mimics classroom ...


In [31]:
embed_size = 300 # how big is each word vector
max_features = None # how many unique words to use (i.e num rows in embedding vector)
maxlen = 30 # max number of words in a question to use #99.99%

## fill up the missing values
X = dataset["treated_question"].values
y = dataset["treated_answer"].values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features, filters='')
tokenizer.fit_on_texts(list(X))

X = tokenizer.texts_to_sequences(X)
y = tokenizer.texts_to_sequences(y)

## Pad the sentences 
X = pad_sequences(X, maxlen=maxlen)
y = pad_sequences(y, maxlen=maxlen)

In [33]:
y[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  2, 21, 18, 65], dtype=int32)

In [34]:
word_index = tokenizer.word_index
max_features = len(word_index)+1
def load_glove(word_index):
    EMBEDDING_FILE = 'glove.6B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if o.split(" ")[0] in word_index)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = 'wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100 and o.split(" ")[0] in word_index )

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [35]:
embedding_matrix_1 = load_glove(word_index)
embedding_matrix_2 = load_fasttext(word_index)
embedding_matrix = np.mean((embedding_matrix_1, embedding_matrix_2), axis=0)  
del embedding_matrix_1, embedding_matrix_2
gc.collect()
np.shape(embedding_matrix)

(72, 300)

In [36]:
questions = dataset['treated_question'].values
answers = dataset['treated_answer'].values

In [37]:
def createVocab(lines, all_eng_words):
    for line in lines:
        for word in line.split(' '):
            if word not in all_eng_words:
                all_eng_words.add(word)
    return all_eng_words

In [40]:
all_questions = set()
all_answers = set()
all_questions = createVocab(questions, all_questions)
all_answers = createVocab(answers, all_answers)

In [60]:
li = []
for line in questions:
    li.append(line.split(' '))
    print(len(line.split(' ')))

8
8
13
15
12
11
7
8
9
11
7
7
7
8
11
9
10


In [61]:
input_words = sorted(list(all_questions))
target_words = sorted(list(all_answers))
num_encoder_tokens = len(all_questions)
num_decoder_tokens = len(all_answers)

input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])

encoder_input_data = np.zeros((len(questions), 15),dtype='float32')
decoder_input_data = np.zeros((len(answers), 50),dtype='float32')
decoder_target_data = np.zeros((len(answers), 50, num_decoder_tokens),dtype='float32')

In [62]:
for i, (input_text, target_text) in enumerate(zip(questions, answers)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [64]:
encoder_inputs = Input(shape=(None,))
en_x = Embedding(num_encoder_tokens, embed_size, weights = [embedding_matrix])(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
dex = Embedding(num_decoder_tokens, embed_size)
final_dex = dex(decoder_inputs)
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    21600       input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 300)    70800       input_4[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LS

In [65]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=128, epochs=20, validation_split=0.05)
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Train on 16 samples, validate on 1 samples
Epoch 1/20
16/16 [==============================] - 2s 97ms/step - loss: 3.1154 - acc: 0.0000e+00 - val_loss: 3.2666 - val_acc: 0.0200
Epoch 2/20
16/16 [==============================] - 0s 4ms/step - loss: 3.1033 - acc: 0.0400 - val_loss: 3.2545 - val_acc: 0.0600
Epoch 3/20
16/16 [==============================] - 0s 4ms/step - loss: 3.0917 - acc: 0.0613 - val_loss: 3.2394 - val_acc: 0.0800
Epoch 4/20
16/16 [==============================] - 0s 4ms/step - loss: 3.0766 - acc: 0.0575 - val_loss: 3.2206 - val_acc: 0.0800
Epoch 5/20
16/16 [==============================] - 0s 4ms/step - loss: 3.0564 - acc: 0.0538 - val_loss: 3.1967 - val_acc: 0.0600
Epoch 6/20
16/16 [==============================] - 0s 4ms/step - loss: 3.0303 - acc: 0.0512 - val_loss: 3.1687 - val_acc: 0.0600
Epoch 7/20
16/16 [==============================] - 0s 4ms/step - loss: 2.9992 - acc: 0.0512 - val_loss: 3.1382 - val_acc: 0.0600
Epoch 8/20
16/16 [========================